### SNGAN with Fractal Blocks, Spectral Normalization and Self Modulation

SpecNorm in the Discriminator, Self Modulation with BatchNorm in the Generator, Wasserstein Loss with Gradient Penalty.
Discriminator and Generators have Fractal Blocks in them, check /models. The level argument determines how deep the fractal block is, 0 being the default fractal block.

The hyperparameters are from the WGAN-GP paper except for the number of disc_steps, which you can decrease with the inclusion of spectral normalization. 

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from models.fractalnet import FractalNet_Generator, FractalNet_Discriminator

In [ ]:
batch_size_train = 64

def get_indices(dataset,class_name):
    indices =  []
    for i in range(len(dataset.targets)):
        if dataset.targets[i] == class_name:
            indices.append(i)
    return indices

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
idx = get_indices(trainset, 2)

#Use the first one if you only want to produce a particular class
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, num_workers=2, sampler = torch.utils.data.sampler.SubsetRandomSampler(idx))
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, num_workers=2, shuffle = True)

In [ ]:
data_loader = iter(trainloader)
(data, target) = next(data_loader)

print(data.shape)
print(data.max(), data.min())
img = np.transpose(data[0], (1, 2, 0))
plt.imshow((img+1)/2)
plt.show()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

noise_dim = 128
netG = FractalNet_Generator(image_size = (32,32), level = 1, noise_dim = noise_dim).to(device)
netD = FractalNet_Discriminator(image_size = (32,32), level = 1, mbd_features = None).to(device)

print(netG)
print("******************************************************************************************************************")
print(netD)

if torch.cuda.device_count() > 1:
    netG = nn.DataParallel(netG, list(range(torch.cuda.device_count())))
    netD = nn.DataParallel(netD, list(range(torch.cuda.device_count())))

#Optional orthogonal initialization of weights, does not work with Spectral Normalization!#########
for m in netG.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear, nn.ConvTranspose2d)):
        nn.init.orthogonal_(m.weight)
'''
for m in netD.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear, nn.ConvTranspose2d)):
        nn.init.orthogonal_(m.weight)
'''

#Two Timescale Update Rule
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.0, 0.9))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0001, betas = (0.0, 0.9))

In [ ]:
epochs = 500
disc_steps = 1
path = "./saved_models/"

for epoch in range(epochs):
    for i, (data, target) in enumerate(trainloader):

        #Dealing with the discriminator################################
        #Specify number of disc updates above##############
        for s in range(disc_steps):
            netD.zero_grad()
        
            real_images = data.to(device)
            b_size = real_images.size(0)
        
            output = netD(real_images).view(-1)
            errD_real = -torch.mean(output)
            errD_real.backward()
            D_x = output.mean().item()

            noise = torch.randn(b_size, noise_dim, device = device)
            fake = netG(noise)

            output = netD(fake.detach()).view(-1)
            errD_fake = torch.mean(output)
            errD_fake.backward()
            D_G_z1 = output.mean().item()

            GP = GradientPenalty(netD, real_images, fake, gp_weight = 10)
            GP.backward()

            errD = errD_fake + errD_real + GP
            optimizerD.step()
            
        #Dealing with the generator###################################
        netG.zero_grad()

        output = netD(fake).view(-1)
        errG = -torch.mean(output)

        D_G_z2 = output.mean().item()
        errG.backward()
        optimizerG.step()

        if i%100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                    % (epoch, epochs, i, len(trainloader),
                    errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
    
    #if epoch%100 == 0:
        #torch.save(netG.state_dict(), path + "sngan_res_cifar_G.pth")
        #torch.save(netD.state_dict(), path + "sngan_res_cifar_D.pth")

In [ ]:
f, a = plt.subplots(4, 8, figsize=(20, 20))
for i in range(8):
  noise = torch.randn(4, noise_dim, device = device)
  with torch.no_grad():
    fake = netG(noise)

  for j in range(4):
      img = fake[j].cpu()
      img = np.transpose(img, (1, 2, 0))
      img = (img+1)/2
      a[j][i].imshow(img)
      a[j][i].axis("off")
    
plt.savefig("CIFCol.png")
plt.show()

plt.imshow(img)
plt.show()